In [11]:
import mysql.connector as sql
import pandas as pd

db_connection = sql.connect(host='127.0.0.1', database='data_transfer', user='root', password='admin', auth_plugin='mysql_native_password')
db_cursor = db_connection.cursor()
db_cursor.execute("SELECT id, datetime, department_id, job_id FROM employee where (datetime BETWEEN '2021-01-01 00:00:00' AND '2021-12-31 23:59:59')")
table_rows = db_cursor.fetchall()

dfEmployees = pd.DataFrame(table_rows, columns=db_cursor.column_names)

In [12]:
dfEmployees

,id,datetime,department_id,job_id
0,1,2021-11-07 02:48:42,2,96
1,3,2021-09-01 23:27:38,5,52
2,4,2021-10-01 13:04:21,12,71
3,5,2021-10-10 22:22:17,6,80
4,6,2021-04-23 23:45:42,6,95
...,...,...,...,...
1654,1993,2021-04-11 12:51:33,7,80
1655,1995,2021-08-22 21:57:53,2,83
1656,1996,2021-04-01 17:48:42,3,23
1657,1997,2021-05-04 12:19:50,2,132


In [6]:
db_cursor.execute('SELECT * FROM department')
table_rows = db_cursor.fetchall()

dfDepartment = pd.DataFrame(table_rows, columns=db_cursor.column_names)
dfDepartment

,id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
5,6,Human Resources
6,7,Services
7,8,Support
8,9,Marketing
9,10,Training


In [7]:
db_cursor.execute('SELECT * FROM job')
table_rows = db_cursor.fetchall()

dfJob = pd.DataFrame(table_rows, columns=db_cursor.column_names)
dfJob

,id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing
...,...,...
178,179,Software Engineer II
179,180,Statistician IV
180,181,Programmer Analyst I
181,182,Account Representative I


In [13]:
dfDepartment.rename(columns = {'id': 'department_id'}, inplace = True)
dfJob.rename(columns = {'id': 'job_id'}, inplace = True)

In [17]:
dfEmployees = dfEmployees.merge(dfDepartment, on = 'department_id')
dfEmployees = dfEmployees.merge(dfJob, on = 'job_id')
dfEmployees

,id,datetime,department_id,job_id,department,job
0,1,2021-11-07 02:48:42,2,96,Sales,Health Coach III
1,3,2021-09-01 23:27:38,5,52,Engineering,Structural Analysis Engineer
2,4,2021-10-01 13:04:21,12,71,Accounting,Statistician II
3,5,2021-10-10 22:22:17,6,80,Human Resources,Quality Engineer
4,6,2021-04-23 23:45:42,6,95,Human Resources,Web Designer II
...,...,...,...,...,...,...
1654,1993,2021-04-11 12:51:33,7,80,Services,Quality Engineer
1655,1995,2021-08-22 21:57:53,2,83,Sales,Cost Accountant
1656,1996,2021-04-01 17:48:42,3,23,Research and Development,Administrative Officer
1657,1997,2021-05-04 12:19:50,2,132,Sales,Staff Accountant III


In [19]:
dfEmployees['quarter'] = pd.PeriodIndex(dfEmployees['datetime'], freq='Q')
dfEmployees

,id,datetime,department_id,job_id,department,job,quarter
0,1,2021-11-07 02:48:42,2,96,Sales,Health Coach III,2021Q4
1,3,2021-09-01 23:27:38,5,52,Engineering,Structural Analysis Engineer,2021Q3
2,4,2021-10-01 13:04:21,12,71,Accounting,Statistician II,2021Q4
3,5,2021-10-10 22:22:17,6,80,Human Resources,Quality Engineer,2021Q4
4,6,2021-04-23 23:45:42,6,95,Human Resources,Web Designer II,2021Q2
...,...,...,...,...,...,...,...
1654,1993,2021-04-11 12:51:33,7,80,Services,Quality Engineer,2021Q2
1655,1995,2021-08-22 21:57:53,2,83,Sales,Cost Accountant,2021Q3
1656,1996,2021-04-01 17:48:42,3,23,Research and Development,Administrative Officer,2021Q2
1657,1997,2021-05-04 12:19:50,2,132,Sales,Staff Accountant III,2021Q2


In [25]:
dfQuarters = dfEmployees.pivot_table(values = 'id', index = ['department', 'job'], columns = 'quarter', aggfunc = 'count').reset_index()
dfQuarters = dfQuarters.rename_axis(None, axis=1)
dfQuarters

,department,job,2021Q1,2021Q2,2021Q3,2021Q4
0,Accounting,Account Representative IV,1.0,NaN,NaN,NaN
1,Accounting,Actuary,NaN,1.0,NaN,NaN
2,Accounting,Analyst Programmer,NaN,NaN,1.0,NaN
3,Accounting,Budget/Accounting Analyst III,NaN,1.0,NaN,NaN
4,Accounting,Cost Accountant,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...
933,Training,Teacher,NaN,2.0,NaN,NaN
934,Training,Technical Writer,NaN,NaN,1.0,NaN
935,Training,VP Product Management,1.0,NaN,NaN,NaN
936,Training,VP Quality Control,NaN,1.0,NaN,NaN


In [28]:
npfltMean_Count_Hired = dfEmployees.groupby(['department']).size().mean()

In [29]:
dfDepartment_Count = dfEmployees.groupby(['department_id', 'department']).size().reset_index(name= 'hired')
dfDepartment_Count

,department_id,department,hired
0,1,Product Management,49
1,2,Sales,94
2,3,Research and Development,149
3,4,Business Development,187
4,5,Engineering,207
5,6,Human Resources,204
6,7,Services,202
7,8,Support,217
8,9,Marketing,142
9,10,Training,114


In [35]:
dfDepartment_Count[dfDepartment_Count['hired'] > npfltMean_Count_Hired].sort_values(by = 'hired', ascending = False)

,department_id,department,hired
7,8,Support,217
4,5,Engineering,207
5,6,Human Resources,204
6,7,Services,202
3,4,Business Development,187
2,3,Research and Development,149
8,9,Marketing,142
